In [1]:
import tkinter as tk
from tkinter import ttk
from functions import capture_and_train, recognize_faces
import os

if not os.path.exists('dataset'):
    os.makedirs('dataset')
if not os.path.exists('trainer'):
    os.makedirs('trainer')# Create the main application window
root = tk.Tk()
root.title("Face Recognition System")
root.geometry("700x500")
root.configure(bg="#0a0a0a")  # Darker background for a sleek look

# Style for buttons
style = ttk.Style()

style.configure("TButton",
                font=("Helvetica", 16, "bold"),
                foreground="#f1c40f",
                background="#0a0a0a",
                padding=15,
                borderwidth=1,
                focusthickness=0,
                relief="flat")

style.map("TButton",
          background=[("active", "#0a0a0a"), ("!disabled", "#0a0a0a")],
          foreground=[("pressed", "#f1c40f"), ("active", "#f1c40f")])

# Title label
label = ttk.Label(root, text="Face Recognition System", font=("Helvetica", 24, "bold"),
                  foreground="#f1c40f", background="#0a0a0a")
label.pack(pady=20)

# Buttons for capturing/training and recognition
capture_button = ttk.Button(
    root, text="Capture & Train Faces", command=capture_and_train, style="TButton")
capture_button.pack(pady=30)

recognize_button = ttk.Button(
    root, text="Recognize Faces", command=recognize_faces, style="TButton")
recognize_button.pack(pady=30)

# Start the tkinter main loop
root.mainloop()

Connecting to the database...
Connected to the database.
Connecting to the database...
Connected to the database.
[DEBUG] Extracted embedding for recognition: [0.01320725 0.00085425 0.01833232 ... 0.         0.01016313 0.01431304]
[DEBUG] Calculated distances: {'Mahmoud': 0.6865701, 'mahmoud3': 0.7632001, 'm': 0.7055019, 'mahmoud2': 0.6254175, 'mahmoud4': 0.59609604}
[DEBUG] Closest match: User ID mahmoud4, Distance: 0.5960960388183594
[DEBUG] Extracted embedding for recognition: [0.01468668 0.         0.01191449 ... 0.         0.04559526 0.        ]
[DEBUG] Calculated distances: {'Mahmoud': 0.72394186, 'mahmoud3': 0.7706459, 'm': 0.54727596, 'mahmoud2': 0.6856921, 'mahmoud4': 0.43906182}
[DEBUG] Closest match: User ID mahmoud4, Distance: 0.43906182050704956
[DEBUG] Extracted embedding for recognition: [0.03172491 0.         0.01251005 ... 0.         0.04527283 0.00133072]
[DEBUG] Calculated distances: {'Mahmoud': 0.7383364, 'mahmoud3': 0.8114301, 'm': 0.5852243, 'mahmoud2': 0.70539236